In [115]:
torch.setdefaulttensortype('torch.FloatTensor')
torch.manualSeed(666)

In [116]:
mnist = require 'mnist'

train = mnist.traindataset()
test = mnist.testdataset()

local mean, std

for _,set in ipairs{train, test} do
    set.data = set.data:float()
    
    if not mean then
        mean = train.data:mean()
        std = train.data:std()
    end
    set.data:add(-mean)
    set.data:div(std)
    
    set.label:add(1)
end

collectgarbage()

In [117]:
require 'nn'

net = nn.Sequential()

net:add(nn.SpatialConvolution(1, 64, 5, 5, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())

net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))

net:add(nn.View(64, -1):setNumInputDims(3))
net:add(nn.Mean(3, 3))
net:add(nn.BatchNormalization(64))

net:add(nn.Linear(64, 10))

params, gradParams = net:getParameters()

collectgarbage()

In [118]:
require 'optim'

datasetIdx = 1

optimState = {
    learningRate = 1e-2,
    momentum = 0.9,
    nesterov = true,
    dampening = 0
}

crit = nn.CrossEntropyCriterion()

In [121]:
logger = optim.Logger('/tmp/mnist-acc.log')
logger:setNames{'Training loss'}
logger:style{'-'}
-- logger:showPlot(false)

In [156]:
function fillBatch(batch, labels, set, currIdx, idxs)
    for k = 1,batch:size(1) do
        batch[k]:copy(set.data[idxs and idxs[currIdx] or currIdx])
        labels[k] = set.label[idxs and idxs[currIdx] or currIdx]
        
        currIdx = currIdx + 1
        if currIdx > set.size then currIdx = 1 end
    end
    
    return currIdx
end

In [138]:
batchSize = 128
batch = torch.FloatTensor(batchSize, 1, 28, 28)
labels = torch.IntTensor(batchSize)

function randomShuffle(t)
  for i = 1,#t do
    local j = math.random(i, #t)
    t[i], t[j] = t[j], t[i]
  end
end

idx = {}
for i = 1,train.data:size(1) do
    idx[i] = i
end

for iter = 1,216 do
    local oldIdx = datasetIdx
    datasetIdx = fillBatch(batch, labels, train, datasetIdx, idx)
    if datasetIdx < oldIdx then
        print('New epoch')
        randomShuffle(idx)
        datasetIdx = 1
    end
    
    timer = torch.Timer()
    local loss = crit:forward(net:forward(batch), labels)
    crit:backward(net.output, labels)
    net:zeroGradParameters()
    net:backward(batch, crit.gradInput)
    print(timer:time().real .. ' sec')
    
    local feval = function(x)
        return batchLoss, gradParams
    end

    optim.sgd(feval, params, optimState)
    
    logger:add{loss}
    logger:plot()
    
    collectgarbage() 
end

3.7428669929504 sec	


3.8177711963654 sec	


3.6353871822357 sec	


3.8255701065063 sec	


3.4845149517059 sec	


...

5.0324149131775 sec	


4.7635970115662 sec	


4.8305861949921 sec	


New epoch	


4.8851311206818 sec	


In [170]:
-- Test
require 'xlua'

local testBatchSize = 100
assert(test.size % testBatchSize == 0)

local correctCount = 0
local batchCount = 0
local testDatasetIdx = 1
local batch = torch.FloatTensor(testBatchSize, 1, 28, 28)
local labels = torch.LongTensor(testBatchSize)

repeat
    batchCount = batchCount + 1
    xlua.progress(testBatchSize * batchCount, test.size)
    
    testDatasetIdx = fillBatch(batch, labels, test, testDatasetIdx)
    net:forward(batch)
    local answer = select(2, net.output:max(2))
    correctCount = correctCount + answer:eq(labels):sum()
    
until testDatasetIdx < testBatchSize

print(correctCount / test.size * 100 .. '%')

 [=============================================>]                              Tot: 3m47s | Step: 16 8 m 

99.04%	


In [141]:
net:clearState()
torch.save('mnist-convnet.t7', net)
torch.save('mnist-convnet-optimstate.t7', optimState)